In [ ]:
import numpy as np
import pandas as pd

Activities are the class labels<br>
It is a 6 class classification

In [ ]:
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

Utility function to print the confusion matrix

In [ ]:
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])
    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

Data directory

In [ ]:
DATADIR = 'HAR/UCI'

Raw data signals<br>
Signals are from Accelerometer and Gyroscope<br>
The signals are in x,y,z directions<br>
Sensor signals are filtered to have only body acceleration<br>
excluding the acceleration due to gravity<br>
Triaxial acceleration from the accelerometer is total acceleration

In [ ]:
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

Utility function to read the data from csv file

In [ ]:
def _read_csv(filename) -> object:
    return pd.read_csv(filename, delim_whitespace=True, header=None)

Utility function to load the load

In [ ]:
def load_signals(subset):
    signals_data = []
    for signal in SIGNALS:
        filename: str = f'E:/HAR/UCI/{subset}/Inertial/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        )

    
    return np.transpose(signals_data, (1, 2, 0))

In [ ]:
def load_y(subset):
    
    filename = f'E:/HAR/UCI/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]
    return pd.get_dummies(y).to_numpy()

In [ ]:
def load_data():
    
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')
    return X_train, X_test, y_train, y_test

Importing tensorflow

Loading the train and test data

In [ ]:
X_train, X_test, Y_train, Y_test = load_data()

Utility function to count the number of classes

In [ ]:
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [ ]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

In [ ]:
print(timesteps)
print(input_dim)
print(len(X_train))

Importing tensorflow

In [ ]:
np.random.seed(42)
import tensorflow as tf

In [ ]:
tf.random.set_seed(42)

Configuring a session

In [ ]:
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

Import Keras

In [ ]:
from tensorflow.python.keras import backend as K

In [ ]:
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

Importing libraries

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

Initializing parameters

In [ ]:
epochs = 30
batch_size = 16
n_hidden = 32

Initiliazing the sequential model

In [ ]:
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Compiling the model

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Training the model

In [ ]:
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Confusion Matrix

In [ ]:
print(confusion_matrix(Y_test, model.predict(X_test)))

In [ ]:
score = model.evaluate(X_test, Y_test)

In [ ]:
print(score)